In [2]:
import numpy as np
import pandas as pd
import itertools
import collections
import math
import datetime
import bisect
import copy
import random
import time
import heapq
import scipy
from functools import reduce
from matplotlib import pyplot as plt
import os

In [4]:
path = 'D:/github/ST5188_StatisticalProject/results/earning_filter'
files = os.listdir(path)

In [40]:
from pandas import Series

def Percentage_Profit(X):
    '''
    Return investment strategy profit
    INPUT:
        X: series, strategy to evaluate
    OUTPUT:
        res: int, profit
    '''
    res = 100*((X[-1]/X[0])-1)
    return res

def Profit_Factor(X):
    '''
    Return investment strategy profit factor
    INPUT:
        X: series, strategy to evaluate
    OUTPUT:
        res: int, profit factor
    '''
    up = 0
    down = 0
    for t in range(1,len(X)):
        diff = X[t]-X[t-1]
        if diff > 0:
            up = up + diff
        else:
            down = down - diff
    res = up/down
    return res

def Percentage_Drawdown(X):
    '''
    Return investment strategy percentage drawdown
    INPUT:
        X: series, strategy to evaluate
    OUTPUT:
        res: int, percentage drawdown
    '''
    draw_p = 0
    draw = 0
    old_max = X[0]
    for t in range(1,len(X)):
        if X[t] > old_max:
            old_max = X[t]
        else:
            diff = old_max - X[t]
            if diff > draw:
                draw = diff
                draw_p = diff/old_max
    res = 100*draw_p
    return res

def Recovery_Factor(X):
    '''
    Return investment strategy recovery factor
    INPUT:
        X: series, strategy to evaluate
    OUTPUT:
        res: int, recovery factor
    '''
    total_profit = X[-1] - X[0]
    draw = 0
    old_max = X[0]
    for t in range(1,len(X)):
        if X[t] > old_max:
            old_max = X[t]
        else:
            diff = old_max - X[t]
            if diff > draw:
                draw = diff
    res = total_profit / draw
    return res

def Sharpe_Ratio(X):
    '''
    Return investment strategy sharpe ratio
    INPUT:
        X: series, strategy to evaluate
    OUTPUT:
        res: int, sharpe ratio
    '''
    from pandas import Series
    from numpy import sqrt, std
    
    X_rend = Series(X).pct_change()[1:]
    mu = ( X[-1]/X[0] )**(252/len(X)) - 1
    sigma = sqrt(252)*std(X_rend)
    res = mu / sigma
    return res

def Sortino_Ratio(X):
    '''
    Return investment strategy sortino ratio
    INPUT:
        X: series, strategy to evaluate
    OUTPUT:
        res: int, sortino ratio
    '''
    from numpy import sqrt
    from pandas import Series
    
    X_rend = Series(X).pct_change()[1:]
    downside_risk = X_rend[X_rend < X_rend.mean()].std()
    mu = ( X[-1]/X[0] )**(252/len(X)) - 1
    sigmaD = sqrt(252)*downside_risk
    res = mu / sigmaD
    return res


def Strategy_Evaluation(X, verbose=True, out=False, n_round=3):
    '''
    Evaluate investment strategy according to well-known metrics
    INPUT:
        X: series, strategy to evaluate
        verbose: bool, manages the verbosity. Default=True
        out: bool, if True, return metrics dict as output. Default=False
        n_round: int, number of decimals considered (if <=0, no round).Default=3
    OUTPUT:
        res: dict, metrics of the strategy
    '''
    from numpy import round
    Properties = {'Percentage Profit': Percentage_Profit(X),
            'Profit Factor': Profit_Factor(X),
            'Percentage Drawdown': Percentage_Drawdown(X),
            'Recovery Factor': Recovery_Factor(X),
            'Sharpe Ratio': Sharpe_Ratio(X),
            'Sortino Ratio': Sortino_Ratio(X)}
   
    
    return Properties

In [63]:
file_path = f'{path}/{file}'
with open(file_path, 'r') as file:
       for line in file:
            print(line.strip()[0])
                

D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N
D
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
N


In [95]:
def read_file(file_path):
    vals = [] 
    dates = []
    d = []
    count = 0
    with open(file_path, 'r') as file:
        next(file) 
        for line in file:
            if line.strip()[0] == "2":
                date, value = line.strip().split(maxsplit=1)
                #data.append([date, float(value)])
                vals.append(float(value))
                dates.append(date)
            else:
                if len(vals)>0:
                    series = pd.Series(vals,index=dates)
                    t = Strategy_Evaluation(series, verbose=True, out=False, n_round=3)
                    d.append(t)
                
                    vals = [] 
                    dates = []
    if len(vals)>0:
        series = pd.Series(vals,index=dates)
        t = Strategy_Evaluation(series, verbose=True, out=False, n_round=3)
        d.append(t)
    df =  pd.DataFrame(d)
    stats_df = pd.DataFrame({
                'mean': df.mean(),
                'std': df.std(),
                'wse': df.min()
            }).transpose()
    return stats_df
    

In [88]:
file = files[0] 
read_file(f'{path}/{file}')

,Percentage Profit,Profit Factor,Percentage Drawdown,Recovery Factor,Sharpe Ratio,Sortino Ratio
mean,0.089513,1.340749,0.671155,0.717684,-0.097759,0.414416
std,0.790689,1.225324,0.377702,2.278362,3.776331,9.537142
min,-0.786499,0.442572,0.152418,-0.777349,-4.327433,-14.314749


In [89]:
def add_file_index(df, file_name):
    df.index = pd.MultiIndex.from_product([[file_name], df.index], names=['Method', 'Perfermance'])
    return df

In [90]:
pd.concat({ f'earning_filter_{i/10}':add_file_index(read_file(f'{path}/{files[i]}'),'') for i in range(len(files))})


Percentage Profit  Profit Factor  \
                   Method Perfermance                                     
earning_filter_0.0        mean                  0.089513       1.340749   
                          std                   0.790689       1.225324   
                          min                  -0.786499       0.442572   
earning_filter_0.1        mean                  0.083142       1.257759   
                          std                   0.782582       1.070317   
                          min                  -0.739360       0.439114   
earning_filter_0.2        mean                  0.076744       1.203349   
                          std                   0.783946       0.976351   
                          min                  -0.692223       0.360298   
earning_filter_0.3        mean                  0.070319       1.164814   
                          std                   0.794714       0.903392   
                          min                  -0.645088       0.342837   
earning_filter_0.4        mean                  0.063868       1.134106   
                          std                   0.814493       0.843491   
                          min                  -0.635169       0.335244   
earning_filter_0.5        mean                  0.057390       1.110825   
                          std                   0.842630       0.796184   
                          min                  -0.759403       0.330119   
earning_filter_0.6        mean                  0.050885       1.086950   
                          std                   0.878306       0.741237   
                          min                  -0.887218       0.326427   
earning_filter_0.7        mean                  0.044353       1.067544   
                          std                   0.920627       0.696884   
                          min                  -1.014913       0.323641   
earning_filter_0.8        mean                  0.037793       1.051182   
                          std                   0.968705       0.660490   
                          min                  -1.142487       0.321464   
earning_filter_0.9        mean                  0.031206       1.037161   
                          std                   1.021714       0.630244   
                          min                  -1.269941       0.319716   
earning_filter_1.0        mean                  0.024591       1.025445   
                          std                   1.078912       0.605222   
                          min                  -1.397275       0.318283   
earning_filter_1.1        mean                  0.017948       1.015471   
                          std                   1.139654       0.584526   
                          min                  -1.524489       0.317085   
earning_filter_1.2        mean                  0.011278       1.007334   
                          std                   1.203391       0.567104   
                          min                  -1.651583       0.316070   
earning_filter_1.3        mean                  0.004579       1.000399   
                          std                   1.269658       0.552606   
                          min                  -1.778557       0.315198   
earning_filter_1.4        mean                 -0.002148       0.994554   
                          std                   1.338068       0.540500   
                          min                  -1.905411       0.314442   
earning_filter_1.5        mean                 -0.008903       0.989351   
                          std                   1.408296       0.530152   
                          min                  -2.032145       0.313780   
earning_filter_1.6        mean                 -0.015686       0.984561   
                          std                   1.480074       0.521035   
                          min                  -2.158759       0.313195   
earning_filter_1.7        mean                 -0.022499       0

In [71]:
df = pd.DataFrame({file:read_file(f'{path}/{file}') for file in files}).T

C:\Users\xul14\AppData\Local\Temp\ipykernel_17840\3305129468.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series = pd.Series(vals,index=dates)
C:\Users\xul14\AppData\Local\Temp\ipykernel_17840\3305129468.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series = pd.Series(vals,index=dates)
C:\Users\xul14\AppData\Local\Temp\ipykernel_17840\3305129468.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  series = pd.Series(vals,index=dates)
C:\Users\xul14\AppData\Local\Temp\ipykernel_17840\3305129468.py:15: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype exp

In [96]:
file1 = 'D:/github/ST5188_StatisticalProject/results/benchmark/benchmark_100_port_test_monthly.txt'
file2 = 'D:/github/ST5188_StatisticalProject/results/quaterly/benchmark_100_port_test_quaterly.txt'


names = ["monthly","quaterly"]
files = [file1,file2]
pd.concat({names[i]:add_file_index(read_file(files[i]),'') for i in range(2)})

Percentage Profit  Profit Factor  \
         Method Perfermance                                     
monthly         mean                  0.031206       1.037161   
                std                   1.021714       0.630244   
                wse                  -1.269941       0.319716   
quaterly        mean                 -0.080801       1.081326   
                std                   0.617651       0.570225   
                wse                  -1.169454       0.462507   

                             Percentage Drawdown  Recovery Factor  \
         Method Perfermance                                         
monthly         mean                    1.166719         0.149432   
                std                     0.356855         1.013720   
                wse                     0.737387        -0.905379   
quaterly        mean                    0.670911         0.469488   
                std                     0.444895         1.562517   
                wse                     0.183757        -0.745792   

                             Sharpe Ratio  Sortino Ratio  
         Method Perfermance                               
monthly         mean            -0.530560      -0.464355  
                std              3.433618       5.793339  
                wse             -6.570996     -11.190404  
quaterly        mean            -0.129583       0.018087  
                std              3.231765       5.922988  
                wse             -4.306970      -8.494166

In [101]:
path = 'D:/github/ST5188_StatisticalProject/results/regression'
files = os.listdir(path)
print(files)
names = ["benchmark","ensemble","svr"]
pd.concat({names[i]:add_file_index(read_file(f'{path}/{files[i]}'),'') for i in range(3)})

['benchmark_100_port_test_monthly.txt', 'Ensemble_statistics.txt', 'SVR_statistics.txt']


Percentage Profit  Profit Factor  \
          Method Perfermance                                     
benchmark        mean                  0.031206       1.037161   
                 std                   1.021714       0.630244   
                 wse                  -1.269941       0.319716   
ensemble         mean                  0.241888       1.415428   
                 std                   0.522592       0.722510   
                 wse                  -0.672929       0.546018   
svr              mean                  0.061986       1.356137   
                 std                   0.531240       0.992385   
                 wse                  -0.673040       0.509087   

                              Percentage Drawdown  Recovery Factor  \
          Method Perfermance                                         
benchmark        mean                    1.166719         0.149432   
                 std                     0.356855         1.013720   
                 wse                     0.737387        -0.905379   
ensemble         mean                    0.516062         0.791344   
                 std                     0.303332         1.421771   
                 wse                     0.174274        -0.702557   
svr              mean                    0.499683         0.963473   
                 std                     0.290066         2.080245   
                 wse                     0.163617        -0.741259   

                              Sharpe Ratio  Sortino Ratio  
          Method Perfermance                               
benchmark        mean            -0.530560      -0.464355  
                 std              3.433618       5.793339  
                 wse             -6.570996     -11.190404  
ensemble         mean             1.272110       2.138571  
                 std              2.875217       5.229664  
                 wse             -3.856029      -7.815810  
svr              mean             0.720369       2.365175  
                 std              3.790671       8.014170  
                 wse             -4.120684      -6.522475